# Gradio Day!

Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [3]:
import gradio as gr # oh yeah!

In [4]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set
Google API Key exists and begins AIzaSyAv


In [5]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [6]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

In [7]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [ ]:
message_gpt("What is today's date?")

## User Interface time!

In [ ]:
# here's a simple function

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

Created My Own Function Named tellmeyourname

In [17]:
# here is additional simple function

def greet(tellmeyourname):
    # print("Hello World"+ name)
    greetings = "Hello "+ tellmeyourname +". Trust you are doing good today"
    return greetings
          

In [ ]:
# shout("hello")
greet("Abhijit")

In [33]:
# The simplicty of gradio. This might appear in "light mode" - I'll show you how to make this in dark mode later.

# gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()
gr.Interface(fn=greet,inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


In [ ]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# Adding inbrowser=True opens up a new browser window automatically

# gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)
gr.Interface(fn=greet, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

In [ ]:
# gr.Interface(fn=greet, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)
gr.Interface(fn=greet, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

## Forcing dark mode

Gradio appears in light mode or dark mode depending on the settings of the browser and computer. There is a way to force gradio to appear in dark mode, but Gradio recommends against this as it should be a user preference (particularly for accessibility reasons). But if you wish to force dark mode for your screens, below is how to do it.

In [ ]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

#  Abhijit wrote this code

In [ ]:
#  Abhijit wrote this code
gr.Interface(fn=greet,
            inputs=[gr.Textbox(label="Write Your Name Here:")],
            outputs=[gr.Textbox(label="Greeting Response:")],
            flagging_mode="never").launch()

# You Give Label to Input & Output Textbox

In [ ]:
# Inputs and Outputs

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

# And now - changing the function from "shout" to "message_gpt"
# Whatever you input in input textbox gets passed to function message_gpt and eventually to ChatGPT

In [ ]:
# And now - changing the function from "shout" to "message_gpt"
# Whatever you input in input textbox gets passed to function message_gpt and eventually to ChatGPT

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [34]:
# Let's use Markdown
# Are you wondering why it makes any difference to set system_message when it's not referred to in the code below it?
# I'm taking advantage of system_message being a global variable, used back in the message_gpt function (go take a look)
# Not a great software engineering practice, but quite sommon during Jupyter Lab R&D!

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [35]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python notebook in week1 folder.

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [36]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [ ]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

## Minor improvement

I've made a small improvement to this code.

Previously, it had these lines:

```
for chunk in result:
  yield chunk
```

There's actually a more elegant way to achieve this (which Python people might call more 'Pythonic'):

`yield from result`

I cover this in more detail in the Intermediate Python notebook in the week1 folder - take a look if you'd like more.

In [37]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [38]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


# Building a company brochure generator

Now you know how - it's simple!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you read the next few cells</h2>
            <span style="color:#900;">
                Try to do this yourself - go back to the company brochure in week1, day5 and add a Gradio UI to the end. Then come and look at the solution.
            </span>
        </td>
    </tr>
</table>

In Python, def __init__(self, url): is the definition of the constructor method of a class, which is automatically called when an object (instance) of that class is created.

Let's break it down:

def: This keyword is used to define a function or method in Python.

__init__: This is a special method in Python known as the constructor. The __init__ method is called when you create a new instance of a class. It's used to initialize the object's attributes (properties) with default or passed-in values.

The double underscores (__) around init are just a naming convention in Python. You’ll see similar methods like __str__ or __repr__ which are also special methods in Python.

self: The first parameter in every instance method, including __init__. It refers to the instance of the class itself. When you create an object, self allows the method to access and modify the object's attributes and call other methods.

url: This is a parameter that you pass to the __init__ method when you create an instance of the Website class. In this case, it represents the URL of the website you want to work with.

Example of How It's Used:
When you create a new Website object like this:

python
Copy
website = Website("https://example.com")
Python will automatically call __init__(self, url) with self referring to the new object and url being the string "https://example.com".

Purpose of __init__(self, url) in Your Code:
This method initializes the Website object.
It takes url as input, sends an HTTP request to that URL to fetch the webpage content, and then processes the webpage to extract the title and body text (after removing non-relevant elements like images, scripts, and styles).
The result of these actions is stored in the object's attributes (self.title and self.text), making the object ready for use.
In short, __init__(self, url) is the method that sets up the Website object with data when you first create it.

In [52]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str
#  A class to represent a website and extract its content.
    
    def __init__(self, url):
        self.url = url
        # Send an HTTP request to the given URL and get the response content
        response = requests.get(url)
        self.body = response.content
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(self.body, 'html.parser')
        # Extract the title of the webpage, or set a default value if not found
        self.title = soup.title.string if soup.title else "No title found"
        # Remove irrelevant elements such as scripts, styles, images, and inputs
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        # Extract and clean the remaining textual content from the webpage
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [53]:
# With massive thanks to Bill G. who noticed that a prior version of this had a bug! Now fixed.

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [54]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [55]:
# Added by Abhijit just to print the website
website_details = Website("https://huggingface.co/")
website_details.get_contents()

'Webpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome to Inference Providers on the Hub 🔥\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\n5 days ago\n•\n498k\n•\n5.3k\ndeepseek-ai/Janus-Pro-7B\nUpdated\n3 days ago\n•\n79.3k\n•\n2.09k\ndeepseek-ai/DeepSeek-V3\nUpdated\n7 days ago\n•\n409k\n•\n2.86k\ntencent/Hunyuan3D-2\nUpdated\n7 days ago\n•\n24.2k\n•\n655\ndeepseek-ai/DeepSeek-R1-Distill-Qwen-32B\nUpdated\n5 days ago\n•\n194k\n•\n750\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.02k\n🌍\nChat With Janus-Pro-7B\nA unified multimodal understanding and generation model.\nRunning\non\nZero\n990\n🌍\n

In [56]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
